# Resolução da lista 3 de NLP

## Alunos:
  - Eduardo Brasil Araujo
  - Gideão Pinheiro"

# Questão 1

In [1]:
import transformers as ppb
import pandas as pd
import numpy as np
import torch

e:\localhost\Desktop\dor-repos\computer-science\natural-language-processing\repo\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

distil_config = ppb.DistilBertConfig(
    max_position_embeddings=1024 * 10,
)

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights, config=distil_config, ignore_mismatched_sizes=True)
model = model_class.from_pretrained(pretrained_weights, config=distil_config, ignore_mismatched_sizes=True)

Some weights of DistilBertModel were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized because the shapes did not match:
- distilbert.embeddings.position_embeddings.weight: found shape torch.Size([512, 768]) in the checkpoint and torch.Size([10240, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
df = pd.read_csv('../datasets/preprocessed_website_classification.csv')

In [4]:
df.dropna(inplace=True)

In [5]:
data = df[['cleaned_website_text', 'Category']]
data = data[data.Category.isin(['Games', 'Food'])]
data = data.sample(frac=1, random_state=101).reset_index(drop=True)

In [11]:
batch_1 = data.rename(columns={'cleaned_website_text': 0, 'Category': 1})[:20]

In [12]:
tokenized = batch_1[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [13]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

In [14]:
np.array(padded).shape

(20, 1100)

In [15]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(20, 1100)

In [16]:
input_ids = torch.tensor(padded, device=torch.device('cuda'))
attention_mask = torch.tensor(attention_mask, device=torch.device('cuda'))

model.to(torch.device('cuda'))

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [17]:
features = last_hidden_states[0][:,0,:].to(torch.device('cpu')).numpy()

In [18]:
labels = batch_1[1]

## Prep

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [20]:
def get_all(y_test, y_pred):
    confusion = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision_1 = precision_score(y_test, y_pred, pos_label='Games')
    recall_1 = recall_score(y_test, y_pred, pos_label='Games')
    f1_1 = f1_score(y_test, y_pred, pos_label='Games')
    precision_2 = precision_score(y_test, y_pred, pos_label='Food')
    recall_2 = recall_score(y_test, y_pred, pos_label='Food')
    f1_2 = f1_score(y_test, y_pred, pos_label='Food')
    return confusion, accuracy, precision_1, recall_1, f1_1, precision_2, recall_2, f1_2

In [21]:
def get_values(y_test, y_pred):
    conf, acc, prec_1, recl_1, f1_1, prec_2, recl_2, f1_2 = get_all(y_test, y_pred)
    print('Confusion:\n', conf)
    print('Accuracy: ', acc)
    print('Precision (Games): ', prec_1)
    print('Recall (Games): ', recl_1)
    print('F1 (Games): ', f1_1)
    print('Precision (Food): ', prec_2)
    print('Recall (Food): ', recl_2)
    print('F1 (Food): ', f1_2)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, train_size=0.7, shuffle=True, random_state=123)

## Logistic Regression

In [23]:
from sklearn.linear_model import LogisticRegression

In [24]:
lr = LogisticRegression(random_state=101).fit(X_train, y_train)

In [25]:
get_values(y_test, lr.predict(X_test))

Confusion:
 [[2 0]
 [4 0]]
Accuracy:  0.3333333333333333
Precision (Games):  0.0
Recall (Games):  0.0
F1 (Games):  0.0
Precision (Food):  0.3333333333333333
Recall (Food):  1.0
F1 (Food):  0.5


e:\localhost\Desktop\dor-repos\computer-science\natural-language-processing\repo\venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Naive-Bayes

In [26]:
from sklearn.naive_bayes import GaussianNB

In [28]:
gnb = GaussianNB().fit(X_train, y_train)

In [29]:
get_values(y_test, gnb.predict(X_test))

Confusion:
 [[2 0]
 [2 2]]
Accuracy:  0.6666666666666666
Precision (Games):  1.0
Recall (Games):  0.5
F1 (Games):  0.6666666666666666
Precision (Food):  0.5
Recall (Food):  1.0
F1 (Food):  0.6666666666666666


## SVM

In [30]:
from sklearn.svm import SVC

In [32]:
svc = SVC().fit(X_train, y_train)

In [33]:
get_values(y_test, svc.predict(X_test))

Confusion:
 [[2 0]
 [4 0]]
Accuracy:  0.3333333333333333
Precision (Games):  0.0
Recall (Games):  0.0
F1 (Games):  0.0
Precision (Food):  0.3333333333333333
Recall (Food):  1.0
F1 (Food):  0.5


e:\localhost\Desktop\dor-repos\computer-science\natural-language-processing\repo\venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Análise dos resultados

Os resultados da matriz de confusão apresentaram-se idênticos no modelo de
regressão logística e SVM, com isso as demais métricas foram as mesmas, mas
o que apresentou melhores resultados foi o de Naive-Bayes, com acurácia maior
que o mínimo aceitável de 50%. O desenvolvimento desta questão se provou
desafiador, uma vez que para treinar o modelo requereu uma quantidade
de memória muito alta; e essa dificuldade se mostrou nos resultados. Provavelmente
uma melhor otimização ou entendimento do modelo poderia melhorar os resultados.

O arranjo com o Naive-Bayes apresenta-se interessante para uma situação em que
é necessário saber com certeza se o texto dado é de fato um texto sobre games,
uma vez que apresentou uma precisão de 100%. Os outros modelos são péssimos em tudo.

# Question 2

In [1]:
from bertopic import BERTopic
import pandas as pd

e:\localhost\Desktop\dor-repos\computer-science\natural-language-processing\repo\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('../datasets/preprocessed_website_classification.csv')

In [3]:
data = df[df['cleaned_website_text'].apply(lambda x: isinstance(x, str))]

In [4]:
data = data.cleaned_website_text

In [5]:
model = BERTopic(language='english')
topics, probs = model.fit_transform(data)

In [6]:
model.get_topic(0)[:10]

[('servic', 0.0121297447773489),
 ('comput', 0.011891495116634771),
 ('bank', 0.011544626187311207),
 ('system', 0.011417894660214879),
 ('group', 0.010892344404359993),
 ('edit', 0.010651425003504127),
 ('scienc', 0.010413544970638464),
 ('learn', 0.01033730741617522),
 ('inform', 0.010331095450289665),
 ('research', 0.00999566298464123)]

In [7]:
model.visualize_topics()

# Question 3

Abaixo está a criação do prompt utilizando o chat-GPT na versão 3.5
A criação do prompt segue o que foi indicado [neste artigo](https://arxiv.org/abs/2304.10428)
Na primeira etapa utilizamos a frase "I am an excelent linguist" para que as respostas sejam dadas utilizando os conhecimentos de linguística. 
Ainda nesta etapa informamos qual a tarefa queremos que seja realizada e informamos que as entidades são do tipo "animal" e após isso indicamos onde estão os exemplos.

Na segunda etapa damos alguns exemplos, fornecendo inputs parecidos com os que ele receberia e outputs que mostram o padrão da resposta.

Após isso, pedimos para que ele complete os outputs e passamos 5 inputs para que ele identifique as categorias.

![GPT-NER](../assets/2023-10-08_11-25.png)

O resultado é mostrado abaixo:

![Resultado](../assets/2023-10-08_11-25_1.png)

# Question 4